In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib
from scipy.optimize import fsolve

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [42]:
## MAKE RIKS DO NOT ERASE!!! 

#specs = loaders.load_density()
#specs = geography.get_coordinate_functions(specs)
#airports = loaders.load_airports(specs)
#travel = loaders.load_travel(airports)['annual']
print('finish loading')

wuhan_R0 = 3
region = 'global'
kappa = 1
infected = 1
wuhan_factor =  wuhan_R0 / airports.loc['WUH', 'density'] # R_0 / density for Wuhan

airports['R0'] = airports.density * wuhan_factor
airports['p_outbreak'] = loaders.calculate_outbreaks(airports, kappa=kappa, n=infected)
travel = loaders.augment_travel(travel, airports) 
rr = travel.query('Origin == "WUH"').sort_values('risk_ij', ascending=False)
drop = ['outgoing_total', 'risk_i','Origin', 'lower','upper','origin_lon','origin_lat',	'dest_lon',	'dest_lat']
rr = rr.drop(drop, axis=1, inplace=False).set_index('Dest', drop=False)

df = pd.read_csv('./data/airports.csv')
df = df.loc[~pd.isnull(df.iata_code)]#.set_index('iata_code')
dic = dict(zip(df.iata_code, df.municipality))
rr['municipality'] = rr.Dest.replace(dic)
dic = dict(zip(df.iata_code, df.name))
rr['name'] = rr.Dest.replace(dic)
dic = dict(zip(df.iata_code, df.iso_region))
rr['region'] = rr.Dest.replace(dic)

regions = pd.read_csv('data/regions.csv')
dic = dict(zip(regions.code, regions.name))
rr['region'] = rr.region.replace(dic)
rr.reset_index(drop=True, inplace=True)
rr.to_csv(f'riskiest_wuhan{wuhan_R0}.csv')

finish loading


In [43]:
rr

,Dest,Prediction,dest_p_outbreak,P_ij,risk_ij,municipality,name,region
0,PEK,511126,0.797243,0.113930,0.090830,Beijing,Beijing Capital International Airport,Beijing Municipality
1,CAN,411599,0.743242,0.091745,0.068189,Guangzhou,Guangzhou Baiyun International Airport,Guangdong Province
2,PVG,342503,0.840701,0.076344,0.064182,Shanghai,Shanghai Pudong International Airport,Shanghai Municipality
3,CTU,222699,0.617717,0.049640,0.030663,Chengdu,Chengdu Shuangliu International Airport,Sichuan Province
4,SZX,271791,0.447037,0.060582,0.027082,Shenzhen,Shenzhen Bao'an International Airport,Guangdong Province
5,TSN,178712,0.580680,0.039835,0.023131,Tianjin,Tianjin Binhai International Airport,Tianjin Municipality
6,XIY,144323,0.502535,0.032170,0.016166,Xi'an,Xi'an Xianyang International Airport,Shaanxi Province
7,XMN,117215,0.542221,0.026127,0.014167,Xiamen,Xiamen Gaoqi International Airport,Fujian Province
8,HGH,88852,0.475972,0.019805,0.009427,Hangzhou,Hangzhou Xiaoshan International Airport,Zhejiang Province
9,NKG,69840,0.578826,0.015567,0.009011,Nanjing,Nanjing Lukou Airport,Jiangsu Province


In [41]:
regions

,id,code,local_code,name,continent,iso_country,wikipedia_link,keywords
0,302811,AD-02,02,Canillo,EU,AD,https://en.wikipedia.org/wiki/Canillo,NaN
1,302812,AD-03,03,Encamp,EU,AD,https://en.wikipedia.org/wiki/Encamp,NaN
2,302813,AD-04,04,La Massana,EU,AD,https://en.wikipedia.org/wiki/La_Massana,NaN
3,302814,AD-05,05,Ordino,EU,AD,https://en.wikipedia.org/wiki/Ordino,NaN
4,302815,AD-06,06,Sant Julià de Lòria,EU,AD,https://en.wikipedia.org/wiki/Sant_Julià_de_Lòria,NaN
...,...,...,...,...,...,...,...,...
3936,306317,ZW-MS,MS,Matabeleland South Province,AF,ZW,https://en.wikipedia.org/wiki/Matabeleland_Sou...,NaN
3937,306318,ZW-MV,MV,Masvingo Province,AF,ZW,https://en.wikipedia.org/wiki/Masvingo_Province,NaN
3938,306319,ZW-MW,MW,Mashonaland West Province,AF,ZW,https://en.wikipedia.org/wiki/Mashonaland_West...,NaN
3939,306320,ZW-U-A,U-A,(unassigned),AF,ZW,NaN,NaN


In [32]:
(YLIM[1]-YLIM[0])/18

8.0